<a href="https://colab.research.google.com/github/neuropython/BioAddMedKnowledgeBase/blob/master/MEG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Magnetoencephalography data analysis

## Dataset description

In this exercise you will be using the ds000117 dataset  entitled  [Integrated Neuroimaging Dataset from Perceptual Tasks](https://www.openfmri.org/dataset/ds000117/) from the OpenfMRI database. This dataset is a comprehensive collection of neuroimaging data obtained from nineteen healthy volunteers, focusing on both functional and structural brain modulations. Functional data include results from electroencephalography (EEG), magnetoencephalography (MEG), and functional magnetic resonance imaging (fMRI). The experiment centred around the performance of a simple perceptual task, where participants were asked to analyse images containing familiar, unfamiliar, and scrambled faces.



## Library description



In this task you will try to analyse MEG data using python library called [MNE](https://mne.tools/stable/index.html). MEG is a non-invasive neuroimaging technique that captures the magnetic fields generated by neural activity. MEG data acquired during experiments are commonly stored in the Elekta/Neuromag Raw Data (.fif) format. This kind of format is capable of storing a variety of data types associated with MEG recordings, including raw sensor data, events, and additional metadata. MEG data is inherently three-dimensional: each sensor (channel) captures magnetic signals over time.

# Let's start!

In [2]:
import pandas as pd
import numpy as np
import mne
import matplotlib.pyplot as plt
%matplotlib inline


ModuleNotFoundError: No module named 'mne'

# Load the MEG data

Load the MEG data sub-01_ses-meg_task-facerecognition_run-01_meg.fif using MNE. Once loaded, extract crucial information, including the sampling frequency, the timestamp of the last recorded sample, the names of the first five channels, and the total number of channels and timesteps.

In [ ]:
raw = mne.io.read_raw_fif('sub-01/ses-meg/meg/sub-01_ses-meg_task-facerecognition_run-01_meg.fif', preload=True)


Now our task is to extract some information about the experiment, let look into the data...

In [ ]:
raw.info

In [ ]:
number_of_time_samples = raw.n_times
time_sec = raw.times
channels_names = raw.ch_names[:5]
number_of_channels = raw.info['nchan']
sampling_frequency = raw.info['sfreq']
last_sample_timestamp = raw.times[-1]

print("Sampling frequency: {} Hz".format(sampling_frequency))
print("Timestamp of the last recorded sample: {:.2f} seconds".format(last_sample_timestamp))
print("Names of the first five channels: {}".format(", ".join(channels_names)))
print("Total number of channels: {}".format(number_of_channels))
print("Total number of timesteps: {}".format(number_of_time_samples))


# Filtering of the signal

Now we will filter a given signal using two different frequency filters (0.1 Hz and 40 Hz) and then visualise the signal before and after. Firstly let's check how the example signal looks like for different channels:

In [ ]:
raw.plot()

Each channel contains a continuous signal from 0 to approximately 500 s. Each of these signals contains responses to different stimuli given at different intervals to the subject. Let's move to step of filtering the the data:

In [ ]:
raw.filter(l_freq=0.1, h_freq=None)

In [ ]:
raw.plot()

The signal is quite noisy, but let's apply high pass filtering and see.

In [ ]:
raw.filter(l_freq=None, h_freq=40)

In [ ]:
raw.plot()

As you probobaly see we used two kinds of filters:

1. **Low-pass Filtering:**
   - *Purpose:* Low-pass filters allow signals with frequencies below a certain cutoff frequency to pass through, attenuating frequencies above that threshold.
   - *Use Cases:*
     - *Remove High-Frequency Noise:* High-frequency noise, such as muscle artifacts or environmental interference, can be effectively reduced by applying a low-pass filter. This is important for extracting meaningful information from the lower-frequency neural signals.
     - *Focus on Slow Components:* In MEG signals, neural activity often occurs at lower frequencies. Applying a low-pass filter can help emphasize these slower components, making it easier to analyze and interpret neural patterns.

2. **High-pass Filtering:**
   - *Purpose:* High-pass filters allow signals with frequencies above a certain cutoff frequency to pass through, attenuating frequencies below that threshold.
   - *Use Cases:*
     - *Remove Low-Frequency Drift:* MEG signals may contain slow baseline drift or low-frequency artifacts. Applying a high-pass filter helps eliminate these low-frequency components, revealing more dynamic and relevant neural activity.
     - *Highlight Fast Changes:* High-pass filtering is useful when researchers are interested in rapid changes in neural activity, such as those associated with cognitive processes or transient responses.

**Overall:**
   - *Noise Reduction:* Both low-pass and high-pass filters contribute to reducing noise in the MEG signal, making it easier to identify and analyze neural patterns.
   - *Signal Enhancement:* By removing unwanted frequency components, filtering enhances the visibility of the desired neural activity, facilitating more accurate analysis and interpretation.

**Sources:**
1. ChatGBT.
2. [Imaging.MRC-CBU: MEG PreProcessing](https://imaging.mrc-cbu.cam.ac.uk/meg/PreProcessing)
3. van Driel, J., Olivers, C. N. L., Fahrenfort, J. J. (2021). "High-pass filtering artifacts in multivariate classification of neural time series data.", J. Neurosci. Methods, 352.ial.
Conclusions

# Let's look closer to choosen channel data

The code below shows example how to draw a plot of MEG data as a function of time for the MEG0143 channel for the first 1200 samples.

In [ ]:
channel_name = "MEG0143"
time = raw.times
start_index= 0
stop_index = 1200
single_channel_signal_cropped = raw.get_data(channel_name, start = start_index, stop = stop_index)
plt.plot(time[start_index:stop_index], single_channel_signal_cropped[0])
plt.title(f'MEG Data for Channel {channel_name}')
plt.xlabel('Time [s]')
plt.ylabel('Magnetic field [fT]')
plt.show()

At accoridng 0.3 [s] we see prominent peak. Let's look into the file sub-01_ses-meg_task-facerecognition_run-01_events.tsv what then happend. There are few first lines from the file:

onset	duration	onset_sample	stim_type	trigger	stim_file  
24.0473	0	26628	Unfamiliar	13	meg/u032.bmp  
27.0473	0	29972	Unfamiliar	14	meg/u032.bmp  
30.2545	0	33390	Unfamiliar	13	meg/u088.bmp  
33.2618	0	36698	Unfamiliar	13	meg/u084.bmp  
36.3536	0	40209	Famous	5	meg/f123.bmp  

There no any matching event, but probably it is the phase of showing images as a testing stage of experiment.
We see that the first trigger (showing an image of unfamiliar face) occured at 24.0473 let's look at this time range:


In [ ]:
channel_name = "MEG0143"
time = raw.times
start_index= 26500
stop_index = 27500
single_channel_signal_cropped = raw.get_data(channel_name, start = start_index, stop = stop_index)
plt.plot(time[start_index:stop_index], single_channel_signal_cropped[0])
plt.title(f'MEG Data for Channel {channel_name}')
plt.xlabel('Time [s]')
plt.ylabel('Magnetic field [fT]')
plt.show()

We also see response to trigger but it is quite noisy. That's why we move to the next step which will be averaging all the responses in order to improve signal to noise ratio.

# Finding events in data, creating epochs of data

As a first step, let's check what triggers/events were found in our data.

In [ ]:
events = mne.find_events(raw)

According to the README for our dataset, the events we are interested in are:

| Trigger  | Label | Simplified Label |
|----------|----------|----------|
| 5 | Initial Famous Face  | FAMOUS |
| 6 | Immediate Repeat Famous Face | FAMOUS |
| 7 | Delayed Repeat Famous Face  | FAMOUS |
| 13 | Initial Unfamiliar Face | UNFAMILIAR |
| 14 | Immediate Repeat Famous Face | UNFAMILIAR |
| 15 | Delayed Repeat Unfamiliar Face | UNFAMILIAR |
| 17 | Initial Scrambled Face  | SCRAMBLED |
| 18 | Immediate Repeat Scrambled Face | SCRAMBLED |
| 19 | Delayed Repeat Scrambled Face | SCRAMBLED |


So we exclude others.
                       

In [ ]:
events = mne.pick_events(events, exclude=[ 256,  261,  262,  263,  269,
  270,  271,  273,  274,  275, 4096, 4101, 4102, 4103, 4109, 4110, 4111, 4113, 4114,
 4115, 4352])

event_dict = {
    "Initial Famous Face": 5,
    "Immediate Repeat Famous Face": 6,
    "Delayed Repeat Famous Face": 7,
    "Initial Unfamiliar Face": 13,
    "Immediate Repeat Unfamiliar Face": 14,
    "Delayed Repeat Unfamiliar Face": 15,
    "Initial Scrambled Face": 17,
    "Immediate Repeat Scrambled Face": 18,
    "Delayed Repeat Scrambled Face": 19}

event_groups = {
    "famous" : ["Initial Famous Face", "Immediate Repeat Famous Face", "Delayed Repeat Famous Face"],
    "unfamiliar" : ["Initial Unfamiliar Face", "Immediate Repeat Unfamiliar Face", "Delayed Repeat Unfamiliar Face"],
    "scrambled": ["Initial Scrambled Face", "Immediate Repeat Scrambled Face", "Delayed Repeat Scrambled Face"]
}




The code below is creating a plot of events in your MEG data, providing information about when certain events occurred during data acquisition. The resulting plot can be useful for understanding the temporal distribution of events in analysed experimental paradigm.

In [ ]:
fig = mne.viz.plot_events(
    events, sfreq=raw.info["sfreq"], first_samp=raw.first_samp, event_id=event_dict
)

### Epochs

In the context of neuroimaging, particularly in EEG (electroencephalography) and MEG (magnetoencephalography) studies, an epoch refers to a segment of continuous data centered around a specific event or marker. These events can be stimuli, responses, or other experimental triggers. The purpose of creating epochs is to segment the data into smaller, analysable time intervals to study neural activity related to particular events.



Code below create epochs from raw MEG data. It employs the mne.Epochs constructor, taking as input the raw MEG data (raw), event information (events), and additional parameters for epoch definition. The tmin parameter is set to -0.2 seconds, indicating the start time of each epoch relative to the onset of events, and tmax is set to 0.6 seconds, indicating the end time. The event_id parameter is provided with a dictionary (event_dict) that maps event labels to their corresponding event codes. The resulting epochs variable holds an Epochs object containing segmented time intervals (epochs) centered around specific events in the MEG data. By setting preload=True, the data is loaded into memory, facilitating faster access for subsequent analyses. These epochs are instrumental for studying and analyzing neural responses related to the specified events, such as computing event-related potentials (ERPs) or conducting further time-frequency analyses.


In [ ]:
epochs = mne.Epochs(raw, events, tmin=-0.2, tmax=0.6, event_id=event_dict, preload=True)

In these two lines of code, an evoked response is computed by averaging the epochs, consolidating the neural activity across multiple trials related to specific events. The evoked.plot(picks='mag') command then generates a visualisation, specifically plotting the averaged magnetic field responses (picks='mag') to provide insight into the temporal dynamics and characteristics of the evoked neural activity recorded by magnetometers in the MEG data. This type of plot is commonly used to analyse and interpret event-related responses in magnetoencephalography experiments.

In [ ]:
evoked = epochs.average()
evoked.plot(picks='mag')

After analysis of the images above, we can see that the largest peaks in the signal occur around 0.170 seconds, so let's draw ourselves a spatial map to look at where the signal was largest:

In [ ]:
fig = evoked.plot_topomap(
    0.17, ch_type="mag", show_names=True, colorbar=False, size=6, res=128
)
fig.suptitle("Visual response")

The intensity of colour corresponds to the magnitude of the signal at a specific time sample, with darker colors representing higher values at 170 ms. Dark red signifies positive values, while dark blue indicates negative values. Both are equally relevant, as we intend to analyse either their absolute values or their negation. Let's plot the average signal for chosen channels for example MEG1611 and MEG1631 (in these channels signal is strong, because they are placed in the dark red area).

In [ ]:
epochs.plot_image(picks=["MEG1611", "MEG1631"])

The signal looks better on the channel and this is what we select for further analysis. Let's take a closer look at what the averaged signal looks like for the various triggers/events for this channel:

In [ ]:
choosen_channel = "MEG1631"
epochs["Initial Famous Face"].plot_image(picks = choosen_channel)

In [ ]:
epochs["Initial Unfamiliar Face"].plot_image(picks = choosen_channel)

In [ ]:
epochs["Initial Scrambled Face"].plot_image(picks = choosen_channel)

Visually, they appear similar, but for the "Initial Scrambled Face" event, the peacock's value seems slightly diminished. Let's check this:


In [ ]:
event_labels = ['Initial Famous Face', 'Initial Unfamiliar Face', 'Initial Scrambled Face']
channel_of_interest = "MEG1631"
latency_magnitude_dict = {}
time_range = (0.0, 0.2)

for event_label in event_labels:
    avg_epoch = epochs[event_label].average()   # Average the epochs for the specific event
    data = avg_epoch.copy().pick([channel_of_interest]).get_data() # Pick the channel of interest and get the data for the specific event
    data *= 1e15     # Multiply the entire signal by 10^15 to change the data into femtoteslas
    time_points = avg_epoch.times     # Get the time points for the averaged epoch
    start_idx, end_idx = np.where((time_points >= time_range[0]) & (time_points <= time_range[1]))[0][[0, -1]]  # Find the start and end indices for the specified time range
    max_time_point = time_points[start_idx + np.argmax(np.abs(data[:, start_idx:end_idx + 1]))]
    max_magnitude = np.max(np.abs(data[:, start_idx:end_idx + 1]))
    latency = max_time_point - time_range[0] # Calculate latency as the difference between the time point and the start of the time range
    latency_magnitude_dict[event_label] = {'latency': latency, 'magnitude': max_magnitude}

for event_label, values in latency_magnitude_dict.items():
    print(f'Event: {event_label}\nLatency: {values["latency"]:.3f} [s] \nMagnitude: {values["magnitude"]:.3f}[fT] \n')


# Tasks

1. Plot the averaged signal for all channels in patient 1 for each trigger ['Initial Famous Face,' 'Initial Unfamiliar Face,' 'Initial Scrambled Face']. Compare this signal with that from the MEG1631 channel and explain why analysing only one channel or a group of several channels might be preferable over analysing the averaged signal from all channels.
2. Perform the same tasks for the remaining four subjects and document the latency and amplitude of the peaks within the specified time range of 0-0.2 seconds, or 0-0.3 seconds if needed for an averaged signal from chosen channel. Channels can be selected manually through visual inspection or by choosing the one with the highest signal amplitude at a given time point.  Present the outcomes of latency and amplitude measurements in a tabular format and calculate the mean and standard deviation an generate a comparision plot.
